In [1]:
import pandas as pd
import numpy as np
import os
import random
import math
import time
from PIL import Image
from PyQt5.QtGui import QColor, QPen, QPixmap
from PyQt5.QtCore import QPointF, QRect
import painter
from utils import QColor_HSV, save, Perlin2D

In [12]:
def draw(width, height, color=700, backgroundColor=(1000,1000,1000), perlinFactorW=2, perlinFactorH=2, step=0.001):
    #for black background set backgroundColor = (0,0,0), for white background set backgroundColor = (1000,1000,1000)
    seed = random.randint(0, 100000000)

    # Set the random seed for repeatability
    np.random.seed(seed)

    p = painter.Painter(width, height)

    # Allow smooth drawing
    p.setRenderHint(p.Antialiasing)

    # Draw the background color
    p.fillRect(1000, 1000, width, height, QColor( *backgroundColor ))

    # Set the pen color
    #p.setPen(QPen(QColor(150, 150, 225, 5), 2))-> more yellow->blue spectrum with mostly blues and greens
    p.setPen(QPen(QColor(0, 0, 0, 5), 2))
    
    print('Creating Noise...')
    p_noise = Perlin2D(width, height, perlinFactorW, perlinFactorH)
    print('Noise Generated!')

    MAX_LENGTH = 2 * width
    STEP_SIZE = step * max(width, height)
    NUM = int(width * height / 1000)
    POINTS = [(random.randint(0, width - 1), random.randint(0, height - 1)) for i in range(NUM)]

    for k, (x_s, y_s) in enumerate(POINTS):
        print(f'{100 * (k + 1) / len(POINTS):.1f}'.rjust(5) + '% Complete', end='\r')

        # The current line length tracking variable
        c_len = 0

        # Actually draw the flow field
        while c_len < MAX_LENGTH:
            # Set the pen color for this segment
            sat = 200 * (MAX_LENGTH - c_len) / MAX_LENGTH
            hue = (color + 130 * (height - y_s) / height) % 360
            p.setPen(QPen(QColor_HSV(hue, sat, 255, 20), 2))

            # angle between -pi and pi
            angle = p_noise[int(x_s), int(y_s)] * math.pi

            # Compute the new point
            x_f = x_s + STEP_SIZE * math.cos(angle)
            y_f = y_s + STEP_SIZE * math.sin(angle)

            # Draw the line
            p.drawLine(QPointF(x_s, y_s), QPointF(x_f, y_f))

            # Update the line length
            c_len += math.sqrt((x_f - x_s) ** 2 + (y_f - y_s) ** 2)

            # Break from the loop if the new point is outside our image bounds
            # or if we've exceeded the line length; otherwise update the point
            if x_f < 0 or x_f >= width or y_f < 0 or y_f >= height or c_len > MAX_LENGTH:
                break
            else:
                x_s, y_s = x_f, y_f

    save(p, fname=f'image_{seed}', folder='.', overwrite=True)

draw(3000, 2000, color=300, perlinFactorW=3, perlinFactorH=5, step=.0005)
#163 is a nice blue/purple, 63 is a blue/yellow/green

Creating Noise...
Noise Generated!


The code above creates *flow fields* using *Perlin noise* which is a classic algorithmic art technique but some people think it is a bit played out, so I decided to try some thing more intriguing: a modified version of the *Voronoi Diagram*

In [ ]:
from scipy.spatial import Voronoi, voronoi_plot_2d, QhullError

import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

from ipywidgets import widgets
from ipywidgets import interact, interact_manual, interactive

In [ ]:
x_bounds = np.array([0, 13])
y_bounds = np.array([0, 16])

x_buffer, y_buffer = 1, 1

x_plot = x_bounds + np.array([x_buffer, -x_buffer])
y_plot = y_bounds + np.array([y_buffer, -y_buffer])

In [ ]:
def make_some_art(num_points=200, percent_to_fill = 0.5, n_fill_lines=5, min_scalar = 0.1,  debug=False, toggle_for_new=False):
    
    x = np.random.uniform(*x_bounds, size=num_points).reshape((num_points, 1))
    y = np.random.uniform(*y_bounds, size=num_points).reshape((num_points, 1))
    pts = np.hstack([x, y])
    
    vor = Voronoi(pts)
    verts = vor.vertices
    shapes_ind = vor.regions
    
    shapes_ind = [s+s[0:1] for s in shapes_ind if len(s)>0 and -1 not in s]
    shapes = [verts[s] for s in shapes_ind]
    
    n_shapes_to_fill = int(percent_to_fill*len(shapes))
    shapes_to_fill = random.sample(shapes, n_shapes_to_fill)
    while any(len(s) != len(shapes_to_fill[0]) for s in shapes_to_fill):
        shapes_to_fill = random.sample(shapes, n_shapes_to_fill)

    fill = []

    for s in shapes_to_fill:
        center = np.mean(s, axis=0)
        for scaler in np.linspace(min_scalar, 1, num=n_fill_lines, endpoint=False):
            scaled = scaler * (s - center) + center
            fill.append(scaled)

    
    fig, ax = plt.subplots(figsize=(20,20))
    ax.set_aspect('equal')
    
    if not debug:
        plt.grid(False)
        plt.axis('off')

    
    ax.set_xlim(*x_plot)
    ax.set_ylim(*y_plot)
    lc = LineCollection(shapes+fill)
    ax.add_collection(lc)
    
    return fig, ax
    
w = interactive(make_some_art,
                num_points=(10,1000,20),
                percent_to_fill=(0., 1., 0.05),
                n_fill_lines=(1, 20, 1),
                min_scalar=(0,1,0.01))
display(w)